In [ ]:
import cv2
import imutils
import numpy as np
import math
import os

import matplotlib.pyplot as plt
%matplotlib qt

white = [255,255,255]

def blank(imagem):
    r, c = imagem.shape
    for i in range(r):
        for j in range(c):
            if (imagem[i][j] != 0):
                return False
            
    return True

def lastPoint(imagem):
    out = imagem.copy()
    while(True):
        res = cv2.erode(out, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
        if blank(res):
            r, c = imagem.shape
            for i in range(r):
                for j in range(c):
                    if (out[i,j]==white).all():
                        return i, j, out
        else:
            out = res
    
def createSeries(path):
    mat = cv2.imread(path)  
    print(path)

    mRows, mCols, mType = mat.shape
    if mRows>mCols:
        mat = imutils.resize(mat, width=300)
    else:
        mat = imutils.resize(mat, height=300)

    img = mat.copy()

    b,g,r = cv2.split(mat)
    linhas, colunas = g.shape
    newTest = g.copy()
    pixel = 0
    for i in range(linhas):
        for j in range(colunas):
        
            pixel = int(g[i,j]) - int(r[i,j])
            if pixel<20:
                newTest[i,j] = 0
            else:
                newTest[i,j] = 2*pixel

    contour = g.copy()
    mask = g.copy()
    contours = cv2.findContours(newTest, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    for i in range(linhas):
        for j in range(colunas):
            contour[i,j] = 0
            mask[i,j] = 0

    cnts = imutils.grab_contours(contours)
    c = max(cnts, key=cv2.contourArea)        
    cv2.drawContours(contour, [c], -1, (255, 0, 0),  2)
    cv2.drawContours(mask, [c], -1, (255, 0, 0),  -1)
    
    x,y,w,h = cv2.boundingRect(c)
    
    cv2.rectangle(contour,(x,y),(x+w,y+h),(255,255,255),2)
    
    
    x = int(x+w/2)
    y = int(y+h/2)

    graph = []
    cont = 0
    red = [255, 0, 0]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    p, = ax.plot(np.linspace(0,1))

    # First display
    plt.show()
    
    plt.draw()
    
    xAxis = []
    for v in c:
        for p in v:
            lines = contour.copy()
            d = int(math.sqrt(math.pow(p[0]-x,2)+math.pow(p[1]-y,2)))
            
            cv2.line(lines, (x, y), (p[0], p[1]), red, 2)
            cv2.imshow("Lines", lines)
            cv2.waitKey(1)
            
            graph.append(d)
            
            xAxis.append(cont)
            
            ax.cla()
            ax.plot(xAxis, graph, color='tab:blue')
            plt.draw()
            #plt.pause(0.05)
            cont +=1
            
    return graph

def myListDir(directory):
    filelist = os.listdir(directory)
    filelist = sorted(filelist)
    return [x for x in filelist
            if not (x.startswith('.'))]

def rollVector(vec):
    maior = vec[0]
    value = 0
    i = 0
    for v in vec:
        if v > maior:
            maior = v
            value = i
        i += 1
    return np.roll(vec, -value)


In [ ]:
trainPath = "/Users/joseluis/Desktop/Database/complete/train" 
with open("/Users/joseluis/Desktop/Database/complete/rollVector.txt", 'w') as f:
    for folders in myListDir(path):
        
        folder = os.path.join(path, folders)
        
        for filename in myListDir(folder):
            file = os.path.join(folder, filename)
            series = createSeries(file)
            rolled = rollVector(series)
            norm = rolled / np.amax(rolled)
            seriesStr = " ".join(str("{0:.2f}".format(i)) for i in norm)
            f.write(seriesStr+"\n")

In [ ]:
validPath = "/Users/joseluis/Desktop/Database/complete/valid" 
with open("/Users/joseluis/Desktop/Database/complete/rollValid.txt", 'w') as f:
    for filename in myListDir(path):
        file = os.path.join(path, filename)
        series = createSeries(file)
        rolled = rollVector(series)
        norm = rolled / np.amax(rolled)
        seriesStr = " ".join(str("{0:.2f}".format(i)) for i in norm)
        f.write(seriesStr+"\n")

In [ ]:
import numpy as np

#/Users/joseluis/Desktop/Database
xTrainFile = open('/Users/joseluis/Desktop/Database/complete/rollVector.txt', 'r')
yTrainFile = open('/Users/joseluis/Desktop/Database/complete/y_train.txt', 'r')

xTestFile = open('/Users/joseluis/Desktop/Database/complete/rollValid.txt', 'r')
yTestFile = open('/Users/joseluis/Desktop/Database/complete/y_valid.txt', 'r')

xTrain = []
yTrain = []
xTest = []
yTest = []

for x in xTrainFile:
    xTrain.append([float(ts) for ts in x.split()])
    
for y in yTrainFile:
    yTrain.append(int(y.rstrip('\n')))
    
for x in xTestFile:
    xTest.append([float(ts) for ts in x.split()])
    
for y in yTestFile:
    yTest.append(int(y.rstrip('\n')))
    
# Convert to numpy for efficiency
samples = np.array(xTrain)
sLabels = np.array(yTrain)
xTest = np.array(xTest)
yTest = np.array(yTest)

# Mapping table for classes
lNames = {1:'Cabernet Franc', 2:'Kekfrankos', 3:'Sargamuskotaly',
          4:'Szurkebarat', 5:'Tramini'}

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('bmh')
%matplotlib inline
plt.figure(figsize=(15,15))
colors = ['#D62728','#2C9F2C','#FD7F23','#1F77B4','#9467BD',
          '#8C564A','#7F7F7F','#1FBECF','#E377C2','#BCBD27',
          '#FFFF00','#FF00FF','#00FF00','#1E90FF','#000000']

for i, r in enumerate(np.arange(len(xTrain))):
    plt.subplot(8,2,i+1)
    plt.plot(xTrain[r], label=lNames[yTrain[r]], color=colors[i], linewidth=2)
    plt.xlabel('Leaf Series')
    plt.legend(loc='upper left')
    plt.tight_layout()

In [ ]:
plt.style.use('bmh')

plt.figure(figsize=(15,10))
colors = ['#D62728','#2C9F2C','#FD7F23','#1F77B4','#9467BD',
          '#8C564A','#7F7F7F','#1FBECF','#E377C2','#BCBD27']

for i, r in enumerate(np.arange(len(xTest))):
    plt.subplot(5,2,i+1)
    plt.plot(xTest[r], label=lNames[yTest[r]], color=colors[i], linewidth=2)
    plt.xlabel('Leaf Series')
    plt.legend(loc='upper left')
    plt.tight_layout()

In [ ]:
import numpy as np

class mykNN:
    def __init__(self, neighbors=5, max_warping_window=10000, subsample_step=1):
        self.neighbors = neighbors
        self.max_warping_window = max_warping_window
        self.subsample_step = subsample_step
    
    def fit(self, v, e, l):
        self.values = v
        self.expected = e
        self.labels = l
    
    #Returns the distance for every point in fit
    def distances(self, x):
        classDist = []
        c=0
        for v in self.values:
            node = []
            dist = fastdtw.fastdtw(v,x)[0]
            node.append(dist)
            node.append(self.expected[c])
            classDist.append(node)
            c += 1
        
        return classDist 
    
    #Returns the ordered chance for every class in Labels
    def chances(self, x):
        near = self.distances(x)
        sortedList = sorted(near,key=lambda x: x[0])
        chances = np.zeros(len(self.labels))
        
        each = 1/self.neighbors*100
        for i in range(self.neighbors):
            j = sortedList[i]
            
            chances[j[1]-1] += each 
        print(chances)  
        return chances
    
    #Returns class and chance of the nearest neighbor
    def predict(self, x):
        nearClass = 0
        nearests = self.chances(x)
        nearChance = nearests[0]
        j = 0
        for c in nearests:
            if c > nearChance:
                nearChance = c
                nearClass = j
            j+=1
        return nearClass+1, nearChance
    
    def predictions(self, m):
        for v in m:
            a, b = self.predict(v)
            print("Sample belongs to class {} with {}% certainty.".format(lNames[a], b))
            
    def validation(self, m, y):
        i = 0
        for v in m:
            a, b = self.predict(v)
            print("Sample identified as class {} with {}% certainty, belongs to class {}."
                  .format(lNames[a], b, lNames[y[i]]))
            i+=1

In [ ]:
#x_train_file = open('/Users/joseluis/Desktop/Database/rollVector.txt', 'r')
#y_train_file = open('/Users/joseluis/Desktop/Database/y_train.txt', 'r')

#x_test_file = open('/Users/joseluis/Desktop/Database/rollValid.txt', 'r')
#y_test_file = open('/Users/joseluis/Desktop/Database/y_valid.txt', 'r')

x_train = []
y_train = []
xTest = []
yTest = []

for x in x_train_file:
    x_train.append([float(ts) for ts in x.split()])
    
for y in y_train_file:
    y_train.append(int(y.rstrip('\n')))
    
for x in x_test_file:
    xTest.append([float(ts) for ts in x.split()])
    
for y in y_test_file:
    yTest.append(int(y.rstrip('\n')))
    
# Convert to numpy for efficiency
samples = np.array(x_train)
sLabels = np.array(y_train)
xTest = np.array(xTest)
yTest = np.array(yTest)

# Mapping table for classes
lNames = {1:'Cabernet Franc', 2:'Kekfrankos', 3:'Sargamuskotaly',
          4:'Szurkebarat', 5:'Tramini'}

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('bmh')
%matplotlib inline
plt.figure(figsize=(11,7))
colors = ['#D62728','#2C9F2C','#FD7F23','#1F77B4','#9467BD',
          '#8C564A','#7F7F7F','#1FBECF','#E377C2','#BCBD27']

for i, r in enumerate([0,3,6,9,12]):
    plt.subplot(3,2,i+1)
    plt.plot(x_train[r], label=lNames[y_train[r]], color=colors[i], linewidth=2)
    plt.xlabel('Leaf Series')
    plt.legend(loc='upper left')
    plt.tight_layout()

In [ ]:
knn = mykNN(neighbors=5)
knn.fit(samples, sLabels, lNames)
knn.validation(xTest, yTest)